In [1]:
import pandas as pd
import time
import datetime
import numpy as np
from pyecharts.charts import Line
from pyecharts.charts import *
from pyecharts.components import Table
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import random
from datetime import datetime,timedelta
from pyecharts.globals import CurrentConfig, OnlineHostType
CurrentConfig.ONLINE_HOST = OnlineHostType.NOTEBOOK_HOST

In [2]:
df = pd.read_csv('.\UserBehavior.csv', 
                 nrows=1000000, header=None, 
                 names=['用户ID','商品ID','商品类目ID','行为类型','时间戳'])
#数据探索
df.info()#查看数据信息
df.describe()#描述性统计
df.describe(include=['O'])
#print (df.head())

#数据清洗
df.drop_duplicates(keep='last',inplace=True)#删除重复值
df.isnull().sum()#查看缺失值
#异常值处理
start_time = time.mktime(time.strptime("2017-11-25 00:00:00",'%Y-%m-%d %H:%M:%S'))
end_time = time.mktime(time.strptime("2017-12-3 23:59:59",'%Y-%m-%d %H:%M:%S'))
df = df[(df.时间戳 >=int(start_time)) & (df.时间戳<= int(end_time))]

#将时间戳转换为datetime格式
df['时间戳'] = df.时间戳.apply(lambda x: datetime.fromtimestamp(x))
df['时间戳'] = pd.to_datetime(df['时间戳'])
#提取时间
df['日期'] = df.时间戳.dt.date
df['时间'] = df.时间戳.dt.time
df['小时'] = df.时间戳.dt.hour
df['星期'] = df.时间戳.dt.weekday
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   用户ID    1000000 non-null  int64 
 1   商品ID    1000000 non-null  int64 
 2   商品类目ID  1000000 non-null  int64 
 3   行为类型    1000000 non-null  object
 4   时间戳     1000000 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 38.1+ MB


,用户ID,商品ID,商品类目ID,行为类型,时间戳,日期,时间,小时,星期
0,1,2268318,2520377,pv,2017-11-25 01:21:10,2017-11-25,01:21:10,1,5
1,1,2333346,2520771,pv,2017-11-25 06:15:33,2017-11-25,06:15:33,6,5
2,1,2576651,149192,pv,2017-11-25 09:21:25,2017-11-25,09:21:25,9,5
3,1,3830808,4181361,pv,2017-11-25 15:04:53,2017-11-25,15:04:53,15,5
4,1,4365585,2520377,pv,2017-11-25 15:49:06,2017-11-25,15:49:06,15,5


In [3]:
#流量分析
#时间维度
#日期维度
pv_daily = df.groupby('日期').count()['用户ID'].rename('pv')
uv_daily = df.groupby('日期')['用户ID'].nunique().rename('uv')
pv_uv_daily = pd.concat([pv_daily,uv_daily],axis = 1)
pv_uv_daily.head()

,pv,uv
日期,,
2017-11-25,104408,6976
2017-11-26,106576,7128
2017-11-27,97808,7026
2017-11-28,99099,7032
2017-11-29,102240,7141


In [4]:
# 点击量&访客量日分布
line = (Line()
       .add_xaxis(pv_uv_daily.index.tolist())
       .add_yaxis('PV', pv_uv_daily['pv'].tolist())
       .add_yaxis('UV', pv_uv_daily['uv'].tolist())
       .set_global_opts(title_opts=opts.TitleOpts(title="点击量&访客量日分布"))
        #标记
       .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
                opts.MarkPointItem(type_="min", name="最小值")])))

line.render_notebook()

In [5]:
#小时维度
pv_hour = df.groupby('小时').count()['用户ID'].rename('pv')
uv_hour = df.groupby('小时')['用户ID'].nunique().rename('uv')
pv_uv_hour = pd.concat([pv_hour,uv_hour],axis = 1)
pv_uv_hour.head()

,pv,uv
小时,,
0,33752,3091
1,15419,1613
2,8920,980
3,6253,700
4,5484,628


In [6]:
line = (Line()
       .add_xaxis(pv_uv_hour.index.tolist())
       .add_yaxis('PV', pv_uv_hour['pv'].tolist())
       .add_yaxis('UV', pv_uv_hour['uv'].tolist())
       .set_global_opts(title_opts=opts.TitleOpts(title="访客量&点击量时段分布"))
        #标记
       .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
                opts.MarkPointItem(type_="min", name="最小值")])))

line.render_notebook()

In [7]:
# 平均访问深度
line = (Line()
       .add_xaxis(pv_uv_daily.index.tolist())
       .add_yaxis('PV', pv_uv_daily['pv']/pv_uv_daily['uv'].tolist())
       .set_global_opts(title_opts=opts.TitleOpts(title="平均访问深度（PV/UV）"))
        #标记
       .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
                opts.MarkPointItem(type_="min", name="最小值")])))

line.render_notebook()

In [8]:
#跳失率
user_count=len(df['用户ID'].unique())
# 计算总的页面浏览数量
pv_count=df[df['行为类型']=='pv'].shape[0]
print('用户浏览的平均访问深度是 %.1f' % (pv_count/user_count))
#每个不同id的用户浏览页面数
pv_count_perUser=df[df['行为类型']=='pv'].groupby('用户ID')['行为类型'].count().reset_index().rename(columns={'行为类型':'pv_count'})
pv_count_perUser.head()
# 计算只浏览过一次界面的用户数量
bounce_user_count=pv_count_perUser[pv_count_perUser['pv_count']==1].shape[0]
#跳失率
print('跳失率是 %.3f%% ' % ((bounce_user_count/user_count)*100))

用户浏览的平均访问深度是 92.0
跳失率是 0.072% 


In [9]:
#用户行为
##获取数据
pv_data=df[df['行为类型']=='pv']
cart_data=df[df['行为类型']=='cart']
fav_data=df[df['行为类型']=='fav']
buy_data=df[df['行为类型']=='buy']

In [10]:
#日期维度 
type_pv_daily = pv_data.groupby(['日期'])['用户ID'].nunique().rename('pv')
type_cart_daily = cart_data.groupby(['日期'])['用户ID'].nunique().rename('cart')
type_fav_daily = fav_data.groupby(['日期'])['用户ID'].nunique().rename('fav')
type_buy_daily = buy_data.groupby(['日期'])['用户ID'].nunique().rename('buy')
type_daily = pd.concat([type_pv_daily,type_cart_daily,type_fav_daily,type_buy_daily],axis = 1)
type_daily.head()

,pv,cart,fav,buy
日期,,,,
2017-11-25,6782,2374,1081,1317
2017-11-26,6928,2479,1178,1324
2017-11-27,6828,2352,1096,1415
2017-11-28,6810,2380,1100,1410
2017-11-29,6930,2404,1173,1467


In [11]:
line = (Line()
       .add_xaxis(type_daily.index.tolist())
       .add_yaxis('PV',type_daily['pv'].tolist())
       .add_yaxis('CART',type_daily['cart'].tolist())
       .add_yaxis('FAV',type_daily['fav'].tolist())
       .add_yaxis('BUY',type_daily['buy'].tolist())
       .set_global_opts(title_opts=opts.TitleOpts(title="行为类型日期分布"))
        #标记
       .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
                opts.MarkPointItem(type_="min", name="最小值")])))

line.render_notebook()

In [12]:
#小时维度 四种行为各自的uv
type_pv_hour = pv_data.groupby(['小时'])['用户ID'].nunique().rename('pv')
type_cart_hour = cart_data.groupby(['小时'])['用户ID'].nunique().rename('cart')
type_fav_hour = fav_data.groupby(['小时'])['用户ID'].nunique().rename('fav')
type_buy_hour =buy_data.groupby(['小时'])['用户ID'].nunique().rename('buy')
type_hour = pd.concat([type_pv_hour,type_cart_hour,type_fav_hour,type_buy_hour],axis = 1)
type_hour.head()

,pv,cart,fav,buy
小时,,,,
0,2984,870,416,384
1,1558,446,184,155
2,942,256,124,91
3,674,184,85,47
4,603,161,82,54


In [13]:
line = (Line()
       .add_xaxis(type_hour.index.tolist())
       .add_yaxis('PV',type_hour['pv'].tolist())
       .add_yaxis('CART',type_hour['cart'].tolist())
       .add_yaxis('FAV',type_hour['fav'].tolist())
       .add_yaxis('BUY',type_hour['buy'].tolist())
       .set_global_opts(title_opts=opts.TitleOpts(title="行为类型时间分布"))
        #标记
       .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
                opts.MarkPointItem(type_="min", name="最小值")])))

line.render_notebook()

In [14]:
#星期维度 四种行为各自的uv
type_pv_week = pv_data.groupby(['星期'])['用户ID'].nunique().rename('pv')
type_cart_week = cart_data.groupby(['星期'])['用户ID'].nunique().rename('cart')
type_fav_week = fav_data.groupby(['星期'])['用户ID'].nunique().rename('fav')
type_buy_week = buy_data.groupby(['星期'])['用户ID'].nunique().rename('buy')
type_week = pd.concat([type_pv_week,type_cart_week,type_fav_week,type_buy_week],axis = 1)
type_week.head()

,pv,cart,fav,buy
星期,,,,
0,6828,2352,1096,1415
1,6810,2380,1100,1410
2,6930,2404,1173,1467
3,7010,2480,1201,1476
4,7054,2565,1179,1418


In [15]:
line = (Line()
       .add_xaxis(type_week.index.tolist())
       .add_yaxis('PV',type_week['pv'].tolist())
       .add_yaxis('CART',type_week['cart'].tolist())
       .add_yaxis('FAV',type_week['fav'].tolist())
       .add_yaxis('BUY',type_week['buy'].tolist())
       .set_global_opts(title_opts=opts.TitleOpts(title="行为类型星期分布"))
        #标记
       .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markpoint_opts=opts.MarkPointOpts(
            data=[
                opts.MarkPointItem(type_="max", name="最大值"),
                opts.MarkPointItem(type_="min", name="最小值")])))

line.render_notebook()

In [16]:
#用户行为转化分析
##用户发生四种行为每个行为的总数
behavior_funnel=df['行为类型'].value_counts().reset_index()
behavior_funnel.columns=['用户行为','访问量']
# 求出单一环节转化率：
temp1 = np.array(behavior_funnel['访问量'][1:])
temp2 = np.array(behavior_funnel['访问量'][0:-1])
single_convs = list(temp1 / temp2)
single_convs.insert(0,1)
single_convs = [round(x,4) for x in single_convs] 
behavior_funnel['单一环节转化率'] = single_convs
#求出总体转化率
temp3 = np.array(behavior_funnel['访问量'])
temp4= np.ones(len(behavior_funnel['访问量'])) * behavior_funnel['访问量'][0]
total_convs = (temp3 / temp4).tolist()
total_convs = [round(x,4) for x in total_convs]
behavior_funnel['总体转化率'] = total_convs
behavior_funnel.head()

,用户行为,访问量,单一环节转化率,总体转化率
0,pv,895636,1.0000,1.0000
1,cart,55447,0.0619,0.0619
2,fav,28088,0.5066,0.0314
3,buy,20359,0.7248,0.0227


In [17]:
#画用户行为总体转化漏斗图
attrs = behavior_funnel['用户行为'].tolist()
values = (np.array(behavior_funnel['总体转化率'])* 100).tolist()
d = [[attrs[i], values[i]] for i in range(len(attrs))]
funnel=(
    Funnel(init_opts=opts.InitOpts(width="800px", height="600px"))
    .add(
        series_name="",
        data_pair=d,
        gap=2,
        label_opts=opts.LabelOpts(is_show=True, position="inside",formatter="{b}"),
        itemstyle_opts=opts.ItemStyleOpts(border_color="#fff", border_width=1),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="用户行为总体转化漏斗图"))
)
funnel.render_notebook()


In [18]:
#留存率分析
# 计算n日留存率
def cal_retention(n):

    # 用于记录出现过的user_id
    user_list=[] 
    # 取最后一天的前N天
    cal_date=pd.Series(df['日期'].unique()).sort_values()[:-n] 
    # 用于存储最后留存率结果
    retention_rates=[] 
    for to_date in cal_date:
        # 通过与已经有记录的用户列表的集合对比，识别新用户
        new_user_list=set(df[df['日期']==to_date]['用户ID'])-set(user_list) 
        # 用于存储最后留存率结果
        user_list.extend(new_user_list) 
        # 第n天留存情况
        user_ndate=df[df['日期']==to_date+timedelta(n)]['用户ID'].unique()
        retention_cnt=0
        for user_id in user_ndate:
            if user_id in new_user_list: 
                retention_cnt+=1
        retention_rate=retention_cnt/len(new_user_list) 
        # 汇总N日留存数据
        retention_rates.append(retention_rate) 
    u_retention=pd.Series(retention_rates,index=cal_date)
    return u_retention

In [19]:
pd.DataFrame({'1':cal_retention(1),
              '2':cal_retention(2),
              '3':cal_retention(3),
              '4':cal_retention(4),
              '5':cal_retention(5),
              '6':cal_retention(6),
              '7':cal_retention(7)})


,1,2,3,4,5,6,7
2017-11-25,0.790998,0.767775,0.764765,0.765052,0.770356,0.772506,0.985235
2017-11-26,0.651553,0.650932,0.665839,0.662733,0.672050,0.979503,0.972671
2017-11-27,0.597424,0.629630,0.677939,0.666667,0.977456,0.974235,NaN
2017-11-28,0.586331,0.687050,0.665468,0.967626,0.985612,NaN,NaN
2017-11-29,0.651685,0.724719,0.955056,0.971910,NaN,NaN,NaN
2017-11-30,0.945946,0.932432,0.972973,NaN,NaN,NaN,NaN
2017-12-01,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN
2017-12-02,1.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#付费率
#根据用户付费率 = 有购买记录的用户 / 活跃用户计算：
paying_user_count=df[df['行为类型']=='buy']['用户ID'].unique().shape[0]
paying_user_count
print('用户付费率PUR %.2f%%' % ( 100 * paying_user_count / user_count))

用户付费率PUR 68.68%


In [21]:
#用户购买次数
user_buy_time=df[df['行为类型']=='buy'].groupby('用户ID')['行为类型'].count().reset_index().rename(columns={'行为类型':'消费次数'})
buy_time_total=user_buy_time.groupby('消费次数')['用户ID'].count().reset_index().rename(columns={'用户ID':'用户数'})
buy_time_total.head()

,消费次数,用户数
0,1,2260
1,2,1574
2,3,1041
3,4,613
4,5,384


In [22]:
bar = (Bar()
       .add_xaxis(buy_time_total.index.tolist())
       .add_yaxis('',buy_time_total.values.tolist())
       .set_global_opts(yaxis_opts=opts.AxisOpts(name='用户数'),
                        xaxis_opts=opts.AxisOpts(name='消费次数'),
                       title_opts=opts.TitleOpts(title="用户消费次数"))
)

bar.render_notebook()

In [23]:
#复购率 = 多次购买用户数 / 总付费用户计算：
#总付费用户：paying_user_count，现在要从buy_time_total表中筛选出购买次数>2的人数总和
payinguser_over_two=buy_time_total[buy_time_total['消费次数']>=2]['用户数'].sum()
print('用户的复购率为 %.2f%%' % (100*(payinguser_over_two/paying_user_count)))

用户的复购率为 66.21%


In [24]:
buyer_retention_diff=df[df['行为类型']=='buy'].groupby('用户ID')['日期'].apply(lambda x:x.sort_values().diff().dropna()).reset_index()
buyer_retention_diff=buyer_retention_diff[['用户ID','日期']]
buyer_retention_diff.describe()

,用户ID,日期
count,1.367000e+04,13670
mean,4.370757e+05,1 days 03:46:54.162399414
std,4.373668e+05,1 days 16:14:28.046351229
min,1.000000e+02,0 days 00:00:00
25%,1.093210e+05,0 days 00:00:00
50%,1.223110e+05,0 days 00:00:00
75%,1.005649e+06,2 days 00:00:00
max,1.017997e+06,8 days 00:00:00


In [25]:
#用户价值分析（RFM)
#拿到用户的购买记录
rfm_data=buy_data.loc[:,['用户ID','商品类目ID','日期']]
#拿到一个月的最大日期
rfm_data['最近日期']=rfm_data['日期'].max()
#计算购买商品距离月底的时间间隔
rfm_data['日期差']=rfm_data['最近日期']-rfm_data['日期']
#转换日期间隔为数字
rfm_data['日期差']=rfm_data['日期差'].apply(lambda x: x.days) 
# 按用户做汇总
rfm_gb = rfm_data.groupby(['用户ID'],as_index=False).agg(
    {'日期差': 'min',  # 计算最近一次消费时间
    '日期': 'count'})# 计算购买频率 
# 重命名列名
rfm_gb.columns=['用户ID','R','F']
rfm_gb.head()
rfm_gb.iloc[:,1:].describe().T

,count,mean,std,min,25%,50%,75%,max
R,6689.0,2.524144,2.397553,0.0,0.0,2.0,4.0,8.0
F,6689.0,3.043654,3.250811,1.0,1.0,2.0,4.0,72.0


In [26]:
# 定义区间边界
r_bins = [-1,2,30] 
f_bins = [0,2,809] 
# rfm 计算
rfm_gb['r_score'] = pd.cut(rfm_gb['R'],r_bins,labels=[i for i in range(len(r_bins)-1,0,-1)])
rfm_gb['f_score'] = pd.cut(rfm_gb['F'],f_bins,labels=[i+1 for i in range(len(f_bins)-1)])


In [27]:
# 计算RFm的得分
# 加权的RFM的值
rfm_gb=rfm_gb.apply(np.int32)
rfm_gb['rfm_score'] = rfm_gb['r_score']*0.6+rfm_gb['f_score']*0.4
#计算rfm的组合
rfm_gb['r_score'] = rfm_gb['r_score'].astype(np.str)
rfm_gb['f_score'] = rfm_gb['f_score'].astype(np.str)
rfm_gb['rfm_group'] = rfm_gb['r_score'].str.cat(rfm_gb['f_score'])
rfm_gb.head()

#按rfm_group统计各个分群的用户数
display_data=rfm_gb.groupby(['rfm_group'],as_index=False)['用户ID'].count()
display_data.columns=['rfm_group','number']
display_data['rfm_group']=display_data['rfm_group'].astype(np.int32)
#求出各个分群占总用户的比例
temp1 = np.array(display_data['number'])
temp2= np.ones(len(display_data['number'])) * display_data['number'].sum()
percent= (temp1 / temp2).tolist()
percent = [round(x,4) for x in percent]
display_data['百分比'] = percent
def func(x):
    if x==11:
        rfm_group='挽留用户'
    if x==12:
        rfm_group='保持用户'
    if x==21:
        rfm_group='发展用户'
    if x==22:
        rfm_group='价值用户'
    return rfm_group
display_data['rfm_group']=display_data['rfm_group'].apply(func)
display_data



,rfm_group,number,百分比
0,挽留用户,2039,0.3048
1,保持用户,803,0.1200
2,发展用户,1795,0.2684
3,价值用户,2052,0.3068


In [28]:
attrs = display_data['rfm_group'].tolist()
values = (np.array(display_data['百分比'])* 100).tolist()
values = [round(x,2) for x in values]
pie = (
    Pie()
    .add(
        "",
        [[attrs[i], values[i]] for i in range(len(attrs))],
        radius=["40%", "55%"],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{b|{b}: }  {per|{d}%}  ",
            background_color="#eee",
            border_color="#aaa",
            border_width=1,
            border_radius=4,
            rich={
                "b": {"fontSize": 16, "lineHeight": 33},
                "per": {
                    "color": "#eee",
                    "backgroundColor": "#334455",
                    "padding": [2, 4],
                    "borderRadius": 2,
                },
            },
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="用户价值分析"))
)
pie.render_notebook()

In [29]:
df.groupby('商品类目ID').count().shape    
item_frequency=buy_data.groupby(['商品类目ID'],as_index=False).count()
item_frequency[item_frequency['商品类目ID'] >=2 ].count()  
item_frequency.count()    

商品类目ID    2513
用户ID      2513
商品ID      2513
行为类型      2513
时间戳       2513
日期        2513
时间        2513
小时        2513
星期        2513
dtype: int64

In [33]:
#pv前10的商品
pv_item=pv_data.groupby('商品类目ID').count().sort_values(by="用户ID" , ascending=False)[:10]
#购买量前10的商品
buy_item=buy_data.groupby('商品类目ID').count().sort_values(by="用户ID" , ascending=False)[:10]
#pv前10和购买量前10中都有的商品
pv_item_list=list(pv_item.index)
buy_item_list=list(buy_item.index)
same_item= list(set(buy_item_list) & set(pv_item_list))
same_item               


[3002561, 1320293, 4756105, 982926, 4801426, 4145813]

In [34]:
#点击量前10商品
bar = (Bar()
       .add_xaxis(buy_item.index.tolist())
       .add_yaxis('',buy_item['用户ID'].tolist())
       .reversal_axis()
       .set_series_opts(label_opts=opts.LabelOpts(position="right"))
       .set_global_opts(title_opts=opts.TitleOpts(title="点击量前十商品"))
      )
bar.render_notebook()

In [35]:
#购买量前10的商品
bar = (Bar()
       .add_xaxis(pv_item.index.tolist())
       .add_yaxis('',buy_item['用户ID'].tolist())
       .reversal_axis()
       .set_series_opts(label_opts=opts.LabelOpts(position="right"))
       .set_global_opts(title_opts=opts.TitleOpts(title="购买量前十商品"))
      )
bar.render_notebook()